# MDPs in TensorFlow - Navigation with Noisy Directions

In this jupyter notebook, we explore MDPs with stochastic transitions in TensorFlow. These stochastic transitions are defined by external noise that is considered as inputs to the MDP recurrent cell.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import abc
import functools
import time

%matplotlib inline

## Modeling MDPs in TensorFlow

All classes defining MDPs must inherit from abstract class ```MDP```.

In [ ]:
class MDP(metaclass=abc.ABCMeta):
    
    @abc.abstractproperty
    def action_size(self):
        return
    
    @abc.abstractproperty
    def state_size(self):
        return

    @abc.abstractmethod
    def transition(self, state, action):
        return

    @abc.abstractmethod
    def reward(self, state, action):
        return


### Navigation in 2D grid with deceleration zone at the center

In [ ]:
class Navigation(MDP):

    def __init__(self, graph, grid, deceleration, max_theta=20):
        self.graph = graph

        self.ndim = grid["ndim"]
        self.max_theta = max_theta

        with self.graph.as_default():

            # grid constants
            self.__size = tf.constant(grid["size"], dtype=tf.float32)
            self.__goal = tf.constant(grid["goal"], dtype=tf.float32)

            # deceleration constants
            self.__center = tf.constant(deceleration["center"], dtype=tf.float32)
            self.__decay  = tf.constant(deceleration["decay"],  dtype=tf.float32)

            # numerical constants
            self.__0_00 = tf.constant(0.00, dtype=tf.float32)
            self.__1_00 = tf.constant(1.00, dtype=tf.float32)
            self.__2_00 = tf.constant(2.00, dtype=tf.float32)
            self.__8_00 = tf.constant(8.00, dtype=tf.float32)

    @property
    def action_size(self):
        return self.ndim
    
    @property
    def state_size(self):
        return self.ndim
        
    def transition(self, state, action, noise):

        with self.graph.as_default():

            # apply rotation noise
            cos = tf.cos(self.max_theta * np.pi / 180 * noise)
            sin = tf.sin(self.max_theta * np.pi / 180 * noise)

            noise_matrix = tf.stack([cos, -sin, sin, cos], axis=1)
            noise_matrix = tf.reshape(noise_matrix, [-1, 2, 2])
            noisy_action = tf.matmul(noise_matrix, tf.reshape(action, [-1, 2, 1]))
            noisy_action = tf.reshape(noisy_action, [-1, 2])

            # distance to center of grid
            d = tf.sqrt(tf.reduce_sum(tf.square(state - self.__center), 1, keep_dims=True))

            # deceleration_factor
            deceleration = self.__2_00 / (self.__1_00 + tf.exp(-self.__decay * d)) - self.__1_00

            # next position
            next_state = state + deceleration * noisy_action
            next_state = tf.clip_by_value(next_state, self.__0_00, self.__size)

        return next_state

    def reward(self, state, action):
        
        with self.graph.as_default():
            # norm L-1 (manhattan distance)
            # r = -tf.reduce_sum(tf.abs(state - self.__goal), 1, keep_dims=True)

            # norm L-2 (euclidean distance)
            r = -tf.sqrt(tf.reduce_sum(tf.square(state - self.__goal), 1, keep_dims=True))

        return r

## Encoding an MDP as a Recurrent Neural Net

### Encapsulate MDP components into RNN cell

In [ ]:
class MDP_RNNCell(tf.nn.rnn_cell.RNNCell):

    def __init__(self, mdp, policy):
        self.mdp = mdp
        self.policy = policy

    @property
    def action_size(self):
        return self.mdp.action_size
        
    @property
    def state_size(self):
        return self.mdp.state_size + 1

    @property
    def output_size(self):
        return self.state_size + self.mdp.action_size + 1

    def __call__(self, inputs, state, scope=None):

        with self.mdp.graph.as_default():

            # add policy network
            mdp_action = self.policy(state)

            # separate MDP state and timestep
            h = tf.unstack(state, axis=1)
            sz = self.mdp.state_size
            mdp_state = tf.stack(h[:sz], axis=1)
            timestep  = tf.reshape(h[sz], [int(state.shape[0]), 1])

            # add MDP components to the RNN cell output
            noise = inputs
            mdp_next_state =  self.mdp.transition(mdp_state, mdp_action, noise)
            mdp_reward = self.mdp.reward(mdp_next_state, mdp_action)

            # gather MDP state and timestep
            mdp_next_state = tf.concat([mdp_next_state, timestep + 1], axis=1)
            
            # concatenate outputs
            outputs = tf.concat([mdp_reward, mdp_next_state, mdp_action], axis=1)

        return outputs, mdp_next_state


### Define the MDP's policy as a Multi-Layer Perceptron (MLP)

In [ ]:
class PolicyNetwork(object):
    
    def __init__(self, graph, layers):
        self.graph = graph
        self.policy = functools.partial(self.__build_network, layers)
    
    def __call__(self, state):
        return self.policy(state)
    
    def __build_network(self, layers, state, limits=1.0):
        assert(layers[0] == state.shape[1])

        with self.graph.as_default():

            with tf.variable_scope('policy'):

                # hidden layers
                outputs = state
                for i, n_h in enumerate(layers[1:]):
                    if i != len(layers)-2:
                        activation = tf.nn.relu
                    else:
                        activation = tf.nn.tanh

                    outputs = tf.layers.dense(outputs,
                                              units=n_h,
                                              activation=activation,
                                              kernel_initializer=tf.glorot_normal_initializer(),
                                              name="layer"+str(i+1))

                # add action limits over last tanh layer
                action = tf.constant(limits) * outputs

            # print(tf.get_default_graph().get_collection('variables'))

        return action

In [ ]:
class MDP_RNN(object):
    
    def __init__(self, mdp, policy, batch_size=1):
        self.cell = MDP_RNNCell(mdp, policy)
        self.batch_size = batch_size
        self.graph = mdp.graph
    
    def unroll(self, inputs, initial_state):

        state_size = self.cell.state_size

        with self.graph.as_default():

            # dynamic time unrolling
            outputs, final_state = tf.nn.dynamic_rnn(
                self.cell,
                inputs,
                initial_state=initial_state,
                dtype=tf.float32)

            # gather reward, state and action series
            outputs = tf.unstack(outputs, axis=2)
            max_time = int(inputs.shape[1])

            reward_series = tf.reshape(outputs[0], [-1, max_time, 1])
            state_series  = tf.stack(outputs[1:1+state_size], axis=2)
            action_series = tf.stack(outputs[1+state_size:],  axis=2)
        
        return reward_series, state_series, action_series, final_state


## Define the Policy Optimizer

In [ ]:
class NoiseGenerator(object):
    
    def __init__(self, batch_size, max_time, ratio):
        self.size = (batch_size, max_time, 1)
        self.noise_size = (int(ratio * batch_size), max_time, 1)

        # no noise at all...
        shape = (batch_size - self.noise_size[0], max_time, 1)
        self.zero_noise = np.zeros(shape=shape).astype(np.float32)

    def __call__(self):
        # sample noise data from normal distribution
        random_noise = np.random.normal(size=self.noise_size).astype(np.float32)
        return np.concatenate([random_noise, self.zero_noise], axis=0)

In [ ]:
class PolicyOptimizer(object):
    
    def __init__(self, graph, loss, total, learning_rate, noise_generator):
        self.graph = graph

        self.loss = loss
        self.total = total
        
        self.noise = noise_generator
        
        # optimization hyperparameters
        self.learning_rate = learning_rate

        with self.graph.as_default():
            # backprop via RMSProp
            self.train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

            # global initializer
            self.init_op = tf.global_variables_initializer()

    def run(self, sess, epoch=100, show_progress=True):
        # initialize variables
        sess.run(self.init_op)
        
        losses = []
        for epoch_idx in range(epoch):
            # generate inputs (noise)
            inputs_data = self.noise()

            # backprop and update weights
            _, loss, total = sess.run([self.train_step, self.loss, self.total], feed_dict={inputs: inputs_data})

            # store and show loss information
            losses.append(loss)
            if show_progress:
                print('Epoch {0:5}: loss = {1}\r'.format(epoch_idx, loss), end='')
        print()
    
        variables = sess.run({ var.name: var for var in tf.trainable_variables() })

        return losses, total, variables


## Putting all components together

In [ ]:
graph = tf.Graph()

### Instantiate the MDP model

In [ ]:
# navigation parameters
grid = {
    'ndim': 2,
    'size': (10.0, 10.0),
    'initial': (1.0,  5.0),
    'goal': (8.0,  5.0)
}

deceleration = {
    'center': (5.0, 5.0),
    'decay': 2.0
}

# MDP model
mdp = Navigation(graph, grid, deceleration)

### Unroll the RNN model

In [ ]:
batch_size = 1000
max_time = 9

with graph.as_default():

    # same initial state for each batch
    x_initial, y_initial = grid['initial']
    x_initial = tf.fill([batch_size], tf.constant(x_initial, tf.float32))
    y_initial = tf.fill([batch_size], tf.constant(y_initial, tf.float32))
    timestep_initial = tf.fill([batch_size], tf.constant(0.0, tf.float32))
    initial_state = tf.stack([x_initial, y_initial, timestep_initial], axis=1)

    # inputs (noise)
    inputs = tf.placeholder(tf.float32, shape=[None, max_time, 1], name="inputs")

# define policy network
layers = [mdp.state_size + 1, 10, 5, mdp.action_size]
policy = PolicyNetwork(graph, layers)

# unroll MDP model
rnn = MDP_RNN(mdp, policy, batch_size)
rewards, states, actions, final_state = rnn.unroll(inputs, initial_state)

### Define the loss function

In [ ]:
with graph.as_default():
    # loss based on total reward
    total = tf.reduce_sum(rewards, 1)
    loss  = tf.reduce_mean(tf.square(total))

### Train the Policy Network

In [ ]:
def train(graph, optimizer, epoch, saver):
    
    # initilize session
    start = time.time()

    with tf.Session(graph=graph) as sess:

        # optimize it, babe!
        losses, total_cost_per_batch, variables = optimizer.run(sess, epoch)
        
        # save model
        save_path = saver.save(sess, 'models/model.ckpt')
        print("Model saved in file: %s" % save_path)
        
    end = time.time()
    uptime = end - start
    print("Done in {0:.6f} sec".format(uptime))

    return losses, total_cost_per_batch, variables, uptime

In [ ]:
# it's time to train...

# hyperparameters
epoch = 300
learning_rate = 0.001

# optimizer
noise_ratio = 1.00
noise_generator = NoiseGenerator(batch_size, max_time, noise_ratio)
optimizer = PolicyOptimizer(graph, loss, total, learning_rate, noise_generator)

# saver
with graph.as_default():
    saver = tf.train.Saver()

# results
losses, total_cost_per_batch, variables, uptime = train(graph, optimizer, epoch, saver)

### Plot loss function and cost per batch

In [ ]:
plt.figure(figsize=(15, 5))

# plotting losses
plt.subplot(121)
plt.plot(losses, 'b-')
plt.xlim(0, epoch)
plt.title('Total Loss')
plt.xlabel("# iterations")
plt.ylabel("total loss")
plt.grid()

# histogram of cumulative cost per batch
plt.subplot(122)
plt.hist(total_cost_per_batch, normed=True, histtype='stepfilled')
plt.title('Distribution of cumulative cost per batch')
plt.xlabel('total')
plt.ylabel('density')

## Evaluate Policy

In [ ]:
def build_initial_states_grid(x_grid_size, y_grid_size, timestep):
    batch_size = x_grid_size * y_grid_size
    x_grid = np.linspace(0.0, grid['size'][0], x_grid_size)
    y_grid = np.linspace(0.0, grid['size'][1], y_grid_size)
    initial_states_grid = []
    for x in x_grid:
        for y in y_grid:
            initial_states_grid.append([x, y, timestep])
    return initial_states_grid, batch_size

In [ ]:
def evaluate_policy(grid_size, timestep):
    graph = tf.Graph()
    
    with graph.as_default():
        initial_states_grid, batch_size = build_initial_states_grid(grid_size, grid_size, timestep)
        initial_state = tf.constant(initial_states_grid)

        with tf.variable_scope('rnn'):
            policy = PolicyNetwork(graph, layers)
            action = policy(initial_state)

        saver = tf.train.Saver()
        with tf.Session(graph=graph) as sess:
            saver.restore(sess, 'models/model.ckpt')
            actions = sess.run(action)
    
    return initial_states_grid, actions

### Plot Policy

In [ ]:
def plot_policy(ax, end, action_grid, actions, timestep, show_deceleration=True):
    
    # params
    xlim, ylim = grid['size']
    xcenter, ycenter = deceleration['center']
    
    # plot configuration
    ax.axis([0.0, xlim, 0.0, ylim])
    ax.set_aspect('equal')
    ax.grid()
    ax.set_title("Policy (step = {})".format(timestep), fontweight="bold", fontsize=16)
    ax.set_xlabel("x coordinate")
    ax.set_ylabel("y coordinate")

    if show_deceleration:
        npoints = 1000
        X, Y = np.meshgrid(np.linspace(0.0, xlim, npoints), np.linspace(0.0, ylim, npoints))
        D = np.sqrt((X - xcenter) ** 2 + (Y - ycenter) ** 2)
        Lambda = 2 / (1 + np.exp(-deceleration['decay'] * D)) - 1.00

        ticks = np.arange(0.0, 1.01, 0.10)
        cp = ax.contourf(X, Y, Lambda, ticks, cmap=plt.cm.bone)
        cp = ax.contour(X, Y, Lambda, ticks, colors='black', linestyles='dashed')
        
    # action_grid
    initial_states_x = [ p[0] for p in action_grid ]
    initial_states_y = [ p[1] for p in action_grid ]
    ax.plot(initial_states_x, initial_states_y, 'g.')
 
    # actions
    ax.quiver(initial_states_x, initial_states_y, actions[:, 0], actions[:, 1],
              angles='xy', scale_units='xy', scale=1, color='dodgerblue', width=0.005,
              label='actions')

    # end
    ax.plot([end[0]], [end[1]], marker='X', markersize=15, color='crimson', label='goal')


In [ ]:
start, end = grid['initial'], grid['goal']

grid_size = 10
timesteps = [0.0, max_time/3, 2*max_time/3, max_time]

fig = plt.figure(figsize=(15, 5))
for i, timestep in enumerate(timesteps):
    ax = fig.add_subplot(1, len(timesteps), i+1)
    initial_states_grid, actions = evaluate_policy(grid_size, int(timestep))
    plot_policy(ax, end, initial_states_grid, actions, int(timestep))

## Simulate policy

In [ ]:
def simulate(graph, series, batch_size=1, max_time=10):
    with graph.as_default():
        saver = tf.train.Saver()

    with tf.Session(graph=graph) as sess:
        # restore learned policy model
        saver.restore(sess, 'models/model.ckpt')

        # sample noise data
        inputs_data = np.random.normal(size=(batch_size, max_time, 1)).astype(np.float32)

        # simulate MDP trajectories
        result = sess.run(series, feed_dict={inputs: inputs_data})
    return result

In [ ]:
graph = tf.Graph()

max_time = 9
batch_size = 3

with graph.as_default():

    # initial states for simulation
    x_start, y_start = grid['initial']
    x_initial = tf.fill([batch_size], tf.constant(x_start, tf.float32))
    y_initial = tf.fill([batch_size], tf.constant(y_start, tf.float32))
    timestep_initial = tf.fill([batch_size], tf.constant(0.0, tf.float32))
    initial_state = tf.stack([x_initial, y_initial, timestep_initial], axis=1)
    delta = [-3.0, -1.5, 1.5, 3.0]
    for delta_y in delta:
        x_initial = tf.fill([batch_size], tf.constant(x_start, tf.float32))
        y_initial = tf.fill([batch_size], tf.constant(y_start + delta_y, tf.float32))
        timestep_initial = tf.fill([batch_size], tf.constant(0.0, tf.float32))
        initial_state = tf.concat([initial_state, tf.stack([x_initial, y_initial, timestep_initial], axis=1)], axis=0)

    batch_size = initial_state.shape[0]

    # inputs
    inputs = tf.placeholder(tf.float32, shape=[batch_size, max_time, 1], name="inputs")

    # MDP model
    mdp = Navigation(graph, grid, deceleration)

    # unroll MDP model
    policy = PolicyNetwork(graph, layers)
    rnn = MDP_RNN(mdp, policy, batch_size)
    rewards, states, actions, final_state = rnn.unroll(inputs, initial_state)

    # simulate
    rewards, states, actions = simulate(graph, [rewards, states, actions], batch_size, max_time)
    
    # check timestep
    for batch in states:
        for i, state in enumerate(batch):
            assert(int(state[2]) == i+1) 


### Plot simulated trajectories

In [ ]:
def plot_simulation(ax, start, end, s_series, a_series, show_deceleration=True, show_initial=False):
    # params
    xlim, ylim = grid['size']
    xcenter, ycenter = deceleration['center']
    
    # plot configuration
    ax.axis([0.0, xlim, 0.0, ylim])
    ax.set_aspect('equal')
    ax.grid()
    ax.set_xlabel("x coordinate")
    ax.set_ylabel("y coordinate")

    if show_deceleration:
        npoints = 1000
        X, Y = np.meshgrid(np.linspace(0.0, xlim, npoints), np.linspace(0.0, ylim, npoints))
        D = np.sqrt((X - xcenter) ** 2 + (Y - ycenter) ** 2)
        Lambda = 2 / (1 + np.exp(-deceleration['decay'] * D)) - 1.00
        ticks = np.arange(0.0, 1.01, 0.10)
        cp = ax.contourf(X, Y, Lambda, ticks, cmap=plt.cm.bone)
        cp = ax.contour(X, Y, Lambda, ticks, colors='black', linestyles='dashed')

    if show_initial:
        initial_states_x = [ p[0] for p in initial_states_grid ]
        initial_states_y = [ p[1] for p in initial_states_grid ]
        ax.plot(initial_states_x, initial_states_y, 'gx')

    # actions
    s_series = [ s[:-1] for s in s_series ]
    positions = np.concatenate([[start], s_series])
    ax.quiver(positions[:-1, 0], positions[:-1, 1], a_series[:, 0], a_series[:, 1],
              angles='xy', scale_units='xy', scale=1, color='dodgerblue', width=0.005,
              label='actions')

    # states
    ax.plot(positions[:, 0], positions[:, 1], '-', marker='o', color='darkblue', markersize=8, label='states')

    # start and end
    ax.plot([start[0]], [start[1]], marker='X', markersize=15, color='limegreen', label='initial')
    ax.plot([end[0]], [end[1]], marker='X', markersize=15, color='crimson', label='goal')


In [ ]:
start, end = grid['initial'], grid['goal']
fig = plt.figure(figsize=(15, 25))
num_plots = int(initial_state.shape[0])
deltas = [0] + delta
rows = len(deltas)
cols = num_plots // len(deltas)
for i in range(num_plots):
    ax = fig.add_subplot(len(deltas),num_plots/len(deltas),i+1)
    idx = i//cols
    start = (grid['initial'][0], grid['initial'][1] + deltas[idx])
    plot_simulation(ax, start, end, states[i], actions[i])